In [1]:
import pandas as pd

In [2]:
#file to load
file_to_load="Resources/purchase_data.csv"

#read file and store into Pandas Data Frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#display the total number of unique players
total_players = len(purchase_data["SN"].unique())
total_players


576

In [4]:
#display the total number of unique in game items
unique_items = len(purchase_data["Item ID"].unique())
unique_items

179

In [5]:
#calculate average price per item
price_average = purchase_data["Price"].mean()
price_average

3.0509871794871795

In [6]:
#total revenue from all in game purchases
total_revenue=purchase_data["Price"].sum()
total_revenue


2379.77

In [7]:
#total number of purchases 
total_purchases=purchase_data["Purchase ID"].count()
total_purchases

780

In [8]:
#Summary Data Frame/cleaned up for organization 
summary_df_1=pd.DataFrame([{"Number of Unique Items": unique_items,
                           "Average Price": price_average,
                           "Number of Purchases":total_purchases,
                           "Total Revenue":total_revenue}])
summary_df_1

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [9]:
#calculate information for gender demographics
gender_data = purchase_data("Gender")
unique_members = gender_grouped["SN"].nunique(
gender_percent = (unique_players/total_players)*100

TypeError: 'DataFrame' object is not callable